# Preprocessing 1
Investment Analysis with Machine Learning\
UROP 1100E (Man Yin Michael YEUNG, 2021 Fall)

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import jieba
import jieba.posseg as pseg
import paddle
import string
import re
from multiprocessing import Pool
from stopwordsiso import stopwords
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
from IPython.display import clear_output

### Read Text Data

In [2]:
text_data = pd.read_parquet('D:/tsclientdata/anatxt.parquet.gzip')

In [3]:
text_data.shape

(1693001, 11)

### Drop Duplicated Rows

In [4]:
text_data = text_data.drop(columns=['FYEAR'])
text_data = text_data.drop_duplicates()

In [5]:
text_data.shape

(598664, 10)

### Preprocess Text Data

In [6]:
text_data.index = text_data.ID; text_data = text_data.drop(columns=['ID'])

In [7]:
text_data = text_data.loc[text_data.content.notna()]

In [8]:
text_data.content = text_data['TITLE'] + text_data.content
text_data = text_data.drop(columns=['TITLE'])

In [9]:
text_data = text_data[['SecuCode','create_date','content']]

In [10]:
text_data = text_data.rename(columns={'create_date':'date'})

### Jieba

In [11]:
chinese_punct = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.． \r\n\t"
numbers = "1234567890"
def remove_punct(list_1):
    return [[''.join(c for c in s[0] if c not in string.punctuation + numbers + chinese_punct + " "),s[1]] for s in list_1]

In [12]:
def filter_emptystr_pos(list_1):
    ret = []; skip_pos = ['eng']; stops = stopwords(["zh"])
    for item in list_1:
        if not item[0] is None and not len(item[0]) == 0 and\
        not str(item[1]) in skip_pos and str(item[0]) not in stops:
            ret.append(item)
    return ret

In [13]:
jieba.enable_paddle()

Find your own paddle version doesn't satisfy the minimum requirement (1.6.1), please install paddle tiny by 'pip install --upgrade paddlepaddle-tiny', or upgrade paddle full version by 'pip install --upgrade paddlepaddle (-gpu for GPU version)' 


In [29]:
print('start:\t',dt.datetime.now())
for yr in range(2010,2021):
    text_data_segment = text_data.loc[text_data.date>=dt.datetime(yr,1,1)]
    text_data_segment = text_data_segment.loc[text_data_segment.date<=dt.datetime(yr,12,31)]
    print(yr,text_data_segment.shape)
    text_data_segment['jieba_content'] = text_data_segment.content.map(lambda x: pseg.cut(x, use_paddle = True))\
    .map(lambda x: [list(y) for y in x]).map(list).map(remove_punct).map(filter_emptystr_pos)
    text_data_segment = text_data_segment[['SecuCode','date','jieba_content','content']]
    text_data_segment.to_parquet('D:/michael/prepocessed_text_segment_'+str(yr)+'.parquet.gzip')
    print('output '+str(yr)+' data',dt.datetime.now())

start:	 2021-10-17 21:42:25.245445
2010 (39901, 3)
output 2010 data 2021-10-17 21:56:51.795762
2011 (46615, 3)
output 2011 data 2021-10-17 22:15:55.024600
2012 (50785, 3)
output 2012 data 2021-10-17 22:33:40.216034
2013 (44700, 3)
output 2013 data 2021-10-17 22:49:49.337383
2014 (42109, 3)
output 2014 data 2021-10-17 23:06:57.808263
2015 (40336, 3)
output 2015 data 2021-10-17 23:25:49.084084
2016 (51046, 3)
output 2016 data 2021-10-17 23:53:57.622747
2017 (59185, 3)
output 2017 data 2021-10-18 00:21:21.720723
2018 (55208, 3)
output 2018 data 2021-10-18 00:46:16.874096
2019 (56735, 3)
output 2019 data 2021-10-18 01:10:35.623762
2020 (61773, 3)
output 2020 data 2021-10-18 01:37:46.290240


In [31]:
# Adding 2021 Data as well
for yr in range(2021,2022):
    text_data_segment = text_data.loc[text_data.date>=dt.datetime(yr,1,1)]
    text_data_segment = text_data_segment.loc[text_data_segment.date<=dt.datetime(yr,12,31)]
    print(yr,text_data_segment.shape)
    text_data_segment['jieba_content'] = text_data_segment.content.map(lambda x: pseg.cut(x, use_paddle = True))\
    .map(lambda x: [list(y) for y in x]).map(list).map(remove_punct).map(filter_emptystr_pos)
    text_data_segment = text_data_segment[['SecuCode','date','jieba_content','content']]
    text_data_segment.to_parquet('D:/michael/prepocessed_text_segment_'+str(yr)+'.parquet.gzip')
    print('output '+str(yr)+' data',dt.datetime.now())

2021 (48647, 3)
output 2021 data 2021-10-18 02:06:57.412099


In [33]:
text_data = pd.read_parquet('D:/michael/prepocessed_text_segment_2010.parquet.gzip')
for yr in range(2011,2022):
    text_data = text_data.append(pd.read_parquet('D:/michael/prepocessed_text_segment_'+str(yr)+'.parquet.gzip'))

text_data['jieba_content'] = text_data.content.map(lambda x: jieba.cut(x, cut_all = False))\
.map(list).map(remove_punct).map(lambda x: list(filter(None, x)))

text_data_1['jieba_content'] = text_data_1.content.map(lambda x: pseg.cut(x, use_paddle = True))\
.map(lambda x: [list(y) for y in x]).map(list).map(remove_punct).map(filter_emptystr_pos)

text_data.jieba_content = text_data.jieba_content.map(lambda x: [e for e in x if e.strip()])

In [41]:
text_data = text_data[['SecuCode','date','jieba_content','content']]
text_data.head(3)

,SecuCode,date,jieba_content,content
ID,,,,
181899,600323,2010-01-04,"[[南海发展, nt], [水价, n], [如期, t], [上调, v], [增厚, v...",南海发展：水价如期上调，增厚业绩约10％ 事项：\r\n\r\n 公司今日公...
181900,002024,2010-01-04,"[[苏宁, ns], [电器, n], [内外, s], [兼修, v], [进入, v],...",苏宁电器：“内外兼修”进入新一轮增长期 投资要点\r\n\r\n 行业前景光...
181901,002092,2010-01-04,"[[泰, ns], [化学, n], [发布, v], [准东, ns], [井田, nr]...",中泰化学：发布准东一井田勘探报告，煤炭开发再推进一步 事件描述：\r\n\r\n ...


In [38]:
text_data.shape

(597040, 4)

In [51]:
text_data.jieba_content = text_data.jieba_content.map(list)

In [52]:
text_data.to_parquet('D:/michael/prepocessed_text_only.parquet.gzip')

In [3]:
text_data = pd.read_parquet('D:/michael/prepocessed_text_only.parquet.gzip',columns=['SecuCode','date','jieba_content'])
text_data.head(3)

,SecuCode,date,jieba_content
ID,,,
181899,600323,2010-01-04,"[[南海发展, nt], [水价, n], [如期, t], [上调, v], [增厚, v..."
181900,002024,2010-01-04,"[[苏宁, ns], [电器, n], [内外, s], [兼修, v], [进入, v],..."
181901,002092,2010-01-04,"[[泰, ns], [化学, n], [发布, v], [准东, ns], [井田, nr]..."


In [4]:
text_data.jieba_content = text_data.jieba_content.map(lambda x: [str(y[0])+str(y[1]) for y in x])

In [5]:
text_data.head(3)

,SecuCode,date,jieba_content
ID,,,
181899,600323,2010-01-04,"[南海发展nt, 水价n, 如期t, 上调v, 增厚v, 业绩n, 约d, 事项n, 公司n..."
181900,002024,2010-01-04,"[苏宁ns, 电器n, 内外s, 兼修v, 进入v, 新一轮nz, 增长期n, 投资vn, ..."
181901,002092,2010-01-04,"[泰ns, 化学n, 发布v, 准东ns, 井田nr, 勘探vn, 报告n, 煤炭n, 开发..."


In [23]:
text_data.to_parquet('D:/michael/prepocessed_text_only_blend.parquet.gzip')

### Read Price Data

In [8]:
price_data = pd.read_parquet('D:/tsclientdata/dsf.parquet.gzip', columns=['SecuCode','date','specret'])

In [9]:
price_data = price_data[price_data['specret'].notna()]
price_data.head()

,SecuCode,date,specret
253867,000876,2001-04-16,-0.015848
253868,000425,2001-04-16,-0.001941
253869,600800,2001-04-16,-0.014478
253870,000831,2001-04-16,-0.009161
253871,600694,2001-04-16,0.001786


In [10]:
price_data.date = price_data.date.map(lambda x: x.strftime('%Y-%m-%d'))

In [11]:
date_list = list(price_data.date.unique())
for dt in range(1,7):
    prev_trade_date_dict = dict()
    for index in range(1,len(date_list)):
        prev_trade_date_dict[date_list[index]] = date_list[index-dt]
    price_data_copy = price_data.copy().loc[price_data.date != date_list[0]]
    price_data_copy.date = price_data_copy.date.map(lambda x: prev_trade_date_dict[x])
    price_data_copy = price_data_copy.rename(columns={'specret':'specret_t+{Dt}'.format(Dt=dt)})
    price_data = pd.merge(price_data, price_data_copy[['SecuCode','date','specret_t+{Dt}'.format(Dt=dt)]],\
                          how = 'inner', on = ['SecuCode','date'])
price_data.head()

,SecuCode,date,specret,specret_t+1,specret_t+2,specret_t+3,specret_t+4,specret_t+5,specret_t+6
0,000876,2001-04-16,-0.015848,0.026132,-0.015259,0.004399,-0.025726,-0.012984,-0.011132
1,000425,2001-04-16,-0.001941,0.036524,0.013614,0.009816,0.029656,0.005790,0.013348
2,600800,2001-04-16,-0.014478,-0.015178,0.003704,0.013818,0.014727,0.045301,-0.007975
3,000831,2001-04-16,-0.009161,-0.011471,0.005515,0.009337,-0.005023,0.002681,-0.039401
4,600694,2001-04-16,0.001786,-0.004309,-0.011374,-0.008585,-0.000338,-0.005862,-0.003602


In [12]:
price_data['specret_2d'] = price_data['specret'] + price_data['specret_t+1']
price_data['specret_tp2-tp6'] = price_data['specret_t+2'] + price_data['specret_t+3'] + price_data['specret_t+4']\
+ price_data['specret_t+5'] + price_data['specret_t+6']
price_data = price_data[['SecuCode','date','specret','specret_2d', 'specret_tp2-tp6']]
price_data.head(3)

,SecuCode,date,specret,specret_2d,specret_tp2-tp6
0,000876,2001-04-16,-0.015848,0.010284,-0.060702
1,000425,2001-04-16,-0.001941,0.034582,0.072224
2,600800,2001-04-16,-0.014478,-0.029656,0.069576


In [13]:
price_data.to_parquet('D:/michael/prepocessed_price_only.parquet.gzip')

### Merge Price and Text Data

In [3]:
price_data = pd.read_parquet('D:/michael/prepocessed_price_only.parquet.gzip')
price_data.head(3)

,SecuCode,date,specret,specret_2d,specret_tp2-tp6
0,000876,2001-04-16,-0.015848,0.010284,-0.060702
1,000425,2001-04-16,-0.001941,0.034582,0.072224
2,600800,2001-04-16,-0.014478,-0.029656,0.069576


In [4]:
text_data = pd.read_parquet('D:/michael/prepocessed_text_only_blend.parquet.gzip')
text_data.head(3)

,SecuCode,date,jieba_content
ID,,,
181899,600323,2010-01-04,"[南海发展nt, 水价n, 如期t, 上调v, 增厚v, 业绩n, 约d, 事项n, 公司n..."
181900,002024,2010-01-04,"[苏宁ns, 电器n, 内外s, 兼修v, 进入v, 新一轮nz, 增长期n, 投资vn, ..."
181901,002092,2010-01-04,"[泰ns, 化学n, 发布v, 准东ns, 井田nr, 勘探vn, 报告n, 煤炭n, 开发..."


In [5]:
text_data.date = text_data.date.map(lambda x: x.strftime('%Y-%m-%d'))
text_data['ID'] = text_data.index

In [6]:
text_data.shape

(597040, 4)

In [7]:
merged_data = pd.merge(text_data, price_data, how='inner', on=['SecuCode','date'])

In [8]:
merged_data.shape

(472693, 7)

In [10]:
merged_data = merged_data[['ID','SecuCode','date','specret','specret_2d','specret_tp2-tp6','jieba_content']]

In [11]:
merged_data.head()

,ID,SecuCode,date,specret,specret_2d,specret_tp2-tp6,jieba_content
0,181899,600323,2010-01-04,0.006658,-0.001522,-0.030300,"[南海发展nt, 水价n, 如期t, 上调v, 增厚v, 业绩n, 约d, 事项n, 公司n..."
1,181900,002024,2010-01-04,-0.004292,-0.022783,-0.047744,"[苏宁ns, 电器n, 内外s, 兼修v, 进入v, 新一轮nz, 增长期n, 投资vn, ..."
2,181901,002092,2010-01-04,0.030815,0.018962,0.055390,"[泰ns, 化学n, 发布v, 准东ns, 井田nr, 勘探vn, 报告n, 煤炭n, 开发..."
3,181939,002092,2010-01-04,0.030815,0.018962,0.055390,"[泰ns, 化学n, 打造v, 新疆ns, 资源n, 平台n, 公司n, 煤矿n, 开发v,..."
4,181902,000568,2010-01-04,-0.033701,-0.045938,-0.036113,"[泸州ns, 老窖n, 公司n, 长期d, 竞争力n, 值得v, 期待v, 白酒n, 行业n..."


In [13]:
merged_data.to_parquet('D:/michael/full_version_data/preprocessed_data_1.parquet.gzip')